In [51]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the Generic Dataset
generic_dataset = pd.read_csv("Generic_Dataset.csv")

# Preprocess Generic Dataset
X_generic = generic_dataset.iloc[:, :20].values
y_generic = generic_dataset.iloc[:, 20].values

# Encode labels
label_encoder_generic = LabelEncoder()
y_generic_encoded = label_encoder_generic.fit_transform(y_generic)

# Split the data into training and testing sets
X_generic_train, X_generic_test, y_generic_train, y_generic_test = train_test_split(
    X_generic, y_generic_encoded, test_size=0.2, random_state=42
)

# Build and train the first AI model
model_generic = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(20,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')  # Adjust the number of classes
])

model_generic.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_generic.fit(X_generic_train, y_generic_train, epochs=25, validation_data=(X_generic_test, y_generic_test))

# Save the model for future use
model_generic.save("model_generic.h5")

# Load the first dataset for parallel AI 1
data_1 = pd.read_csv("data_1.csv")

# Preprocess data_1
X_data_1 = data_1.iloc[:, :14].values
y_data_1 = data_1.iloc[:, 14].values

# Encode labels if needed
label_encoder_data_1 = LabelEncoder()
y_data_1_encoded = label_encoder_data_1.fit_transform(y_data_1)

# Build and train the first parallel AI model
model_parallel_1 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(14,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')  # Adjust the number of classes
])

model_parallel_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_parallel_1.fit(X_data_1, y_data_1_encoded, epochs=50)

# Save the model for future use
model_parallel_1.save("model_parallel_1.h5")

# Load the second dataset for parallel AI 2
data_2 = pd.read_csv("data_2.csv")

# Preprocess data_2
X_data_2 = data_2.iloc[:, :5].values
y_data_2 = data_2.iloc[:, 5].values

# Encode labels if needed
label_encoder_data_2 = LabelEncoder()
y_data_2_encoded = label_encoder_data_2.fit_transform(y_data_2)

# Build and train the second parallel AI model
model_parallel_2 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(5,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')  # Adjust the number of classes
])

model_parallel_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_parallel_2.fit(X_data_2, y_data_2_encoded, epochs=50)

# Save the model for future use
model_parallel_2.save("model_parallel_2.h5")

# Combine the outputs of parallel AI 1 and AI 2 for the third AI
output_parallel_1 = model_parallel_1.predict(X_data_1)
output_parallel_2 = model_parallel_2.predict(X_data_2)

# Choose the minimum number of rows for concatenation
min_rows = min(output_parallel_1.shape[0], output_parallel_2.shape[0])

# Subset the outputs to have the same number of rows
output_parallel_1 = output_parallel_1[:min_rows]
output_parallel_2 = output_parallel_2[:min_rows]

# Concatenate the outputs
combined_output = tf.keras.layers.Concatenate()([output_parallel_1, output_parallel_2])

# Convert to Pandas DataFrames
output_df1 = pd.DataFrame(output_parallel_1)
output_df2 = pd.DataFrame(output_parallel_2)

# Save combined outputs to CSV
output_df1.to_csv("data_watch1.csv", index=False)
output_df2.to_csv("data_watch2.csv", index=False)

# Convert parallel AI 1 output to a single column (scaling down)
scaled_output_parallel_1 = output_parallel_1.argmax(axis=1)

# Convert parallel AI 2 output to a single column (scaling down)
scaled_output_parallel_2 = output_parallel_2.argmax(axis=1)

# Save scaled outputs along with labels to CSV
scaled_output_df1 = pd.DataFrame({'label': y_data_1_encoded[:min_rows], 'scaled_output': scaled_output_parallel_1})
scaled_output_df1.to_csv("scaled_output1.csv", index=False)

scaled_output_df2 = pd.DataFrame({'label': y_data_2_encoded[:min_rows], 'scaled_output': scaled_output_parallel_2})
scaled_output_df2.to_csv("scaled_output2.csv", index=False)

# Save the combined output to CSV
combined_output_df = pd.DataFrame(combined_output.numpy())
combined_output_df.to_csv("combined_output.csv", index=False)

# Define num_classes_data_3 after combining outputs
num_classes_data_3 = 4  # Replace with the actual number of classes for the third dataset

# Build and train the third AI model
model_combined = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(combined_output.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes_data_3, activation='softmax')  # Adjust the number of classes
])

# Load the third dataset for the final AI
data_3 = pd.read_csv("generated_data1.csv")

# Preprocess data_3
X_data_3 = data_3.iloc[:, :-1].values
y_data_3 = data_3.iloc[:, -1].values

# Encode labels if needed
label_encoder_data_3 = LabelEncoder()
y_data_3_encoded = label_encoder_data_3.fit_transform(y_data_3)

# Compile the model
model_combined.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model using the concatenated output of the parallel AIs and the third dataset
model_combined.fit(combined_output, y_data_3_encoded[:min_rows], epochs=100, validation_split=0.2)

# Save the output of the final AI to CSV
final_output = model_combined.predict(combined_output)
final_output_df = pd.DataFrame(final_output)
final_output_df.to_csv("final_output.csv", index=False)  # Save final output to "final_output.csv"

# Save the model for future use
model_combined.save("model_combined.h5")

Epoch 1/25
1/1 [==============================] - 1s 1s/step - loss: 0.4774 - accuracy: 0.7500 - val_loss: 0.9443 - val_accuracy: 0.0000e+00
Epoch 2/25
1/1 [==============================] - 0s 96ms/step - loss: 0.4186 - accuracy: 0.7500 - val_loss: 0.9824 - val_accuracy: 0.0000e+00
Epoch 3/25
1/1 [==============================] - 0s 67ms/step - loss: 0.3663 - accuracy: 1.0000 - val_loss: 1.0215 - val_accuracy: 0.0000e+00
Epoch 4/25
1/1 [==============================] - 0s 82ms/step - loss: 0.3213 - accuracy: 1.0000 - val_loss: 1.0628 - val_accuracy: 0.0000e+00
Epoch 5/25
1/1 [==============================] - 0s 100ms/step - loss: 0.2817 - accuracy: 1.0000 - val_loss: 1.1051 - val_accuracy: 0.0000e+00
Epoch 6/25
1/1 [==============================] - 0s 83ms/step - loss: 0.2464 - accuracy: 1.0000 - val_loss: 1.1478 - val_accuracy: 0.0000e+00
Epoch 7/25
1/1 [==============================] - 0s 99ms/step - loss: 0.2157 - accuracy: 1.0000 - val_loss: 1.1935 - val_accuracy: 0.0000e+00


/Users/apple/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
2/2 [==============================] - 1s 15ms/step - loss: 1.4266 - accuracy: 0.2833
Epoch 2/50
2/2 [==============================] - 0s 49ms/step - loss: 1.3765 - accuracy: 0.3500
Epoch 3/50
2/2 [==============================] - 0s 41ms/step - loss: 1.3374 - accuracy: 0.4500
Epoch 4/50
2/2 [==============================] - 0s 32ms/step - loss: 1.3087 - accuracy: 0.4500
Epoch 5/50
2/2 [==============================] - 0s 27ms/step - loss: 1.2828 - accuracy: 0.5333
Epoch 6/50
2/2 [==============================] - 0s 25ms/step - loss: 1.2609 - accuracy: 0.4500
Epoch 7/50
2/2 [==============================] - 0s 26ms/step - loss: 1.2380 - accuracy: 0.4333
Epoch 8/50
2/2 [==============================] - 0s 16ms/step - loss: 1.2157 - accuracy: 0.4333
Epoch 9/50
2/2 [==============================] - 0s 28ms/step - loss: 1.1951 - accuracy: 0.4500
Epoch 10/50
2/2 [==============================] - 0s 20ms/step - loss: 1.1729 - accuracy: 0.4667
Epoch 11/50
2/2 [============

/Users/apple/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
7/7 [==============================] - 2s 3ms/step - loss: 1.3795 - accuracy: 0.2500
Epoch 2/50
7/7 [==============================] - 0s 4ms/step - loss: 1.3545 - accuracy: 0.5500
Epoch 3/50
7/7 [==============================] - 0s 6ms/step - loss: 1.3317 - accuracy: 0.8450
Epoch 4/50
7/7 [==============================] - 0s 4ms/step - loss: 1.3108 - accuracy: 0.7550
Epoch 5/50
7/7 [==============================] - 0s 4ms/step - loss: 1.2967 - accuracy: 0.3400
Epoch 6/50
7/7 [==============================] - 0s 4ms/step - loss: 1.2753 - accuracy: 0.6550
Epoch 7/50
7/7 [==============================] - 0s 3ms/step - loss: 1.2505 - accuracy: 0.8000
Epoch 8/50
7/7 [==============================] - 0s 4ms/step - loss: 1.2247 - accuracy: 0.9000
Epoch 9/50
7/7 [==============================] - 0s 4ms/step - loss: 1.1967 - accuracy: 0.9200
Epoch 10/50
7/7 [==============================] - 0s 6ms/step - loss: 1.1617 - accuracy: 0.8850
Epoch 11/50
7/7 [======================

/Users/apple/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7/7 [==============================] - 0s 3ms/step
Epoch 1/100
1/2 [==============>...............] - ETA: 1s - loss: 1.3894 - accuracy: 0.3750

ValueError: Data cardinality is ambiguous:
  x sizes: 12
  y sizes: 0
Make sure all arrays contain the same number of samples.

In [46]:
print("Shapes - combined_output:", combined_output.shape, "y_data_3_encoded:", y_data_3_encoded[:min_rows].shape)

Shapes - combined_output: (60, 8) y_data_3_encoded: (48,)


In [47]:
y_data_3_encoded[:min_rows]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [48]:
y_data_3_encoded

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [44]:
combined_output

array([[1.96010265e-02, 6.75910175e-01, 2.68572897e-01, 3.59159559e-02,
        5.91505915e-02, 9.38867629e-01, 6.48360641e-04, 1.33345148e-03],
       [1.18282018e-02, 5.31453252e-01, 4.24060792e-01, 3.26577649e-02,
        1.91286206e-01, 7.67947733e-01, 2.40655411e-02, 1.67004392e-02],
       [1.35117145e-02, 6.43741369e-01, 3.20517898e-01, 2.22289786e-02,
        1.42903477e-01, 8.48916709e-01, 4.13357839e-03, 4.04621195e-03],
       [6.48885791e-04, 8.02819729e-01, 1.95927531e-01, 6.03829511e-04,
        2.73285266e-02, 9.69007432e-01, 2.13875226e-03, 1.52527692e-03],
       [2.19526468e-03, 5.58309793e-01, 4.36713278e-01, 2.78157671e-03,
        2.16302633e-01, 7.70959079e-01, 1.18373113e-03, 1.15545187e-02],
       [2.96774809e-03, 8.01086545e-01, 1.92909598e-01, 3.03610181e-03,
        8.60853493e-02, 8.34195554e-01, 6.39602020e-02, 1.57589857e-02],
       [1.51705847e-03, 8.17519248e-01, 1.77347064e-01, 3.61656561e-03,
        2.29085252e-01, 7.36757815e-01, 9.63932928e-03, 2.

In [30]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# Load the input data for prediction
input_data_generic = [2, 3, 3, 0, 2, 0, 2, 1, 2, 0, 1, 2, 0, 0, 2, 0, 0, 2, 1, 0]
input_data_parallel_1 = [0, 0.66, 0, 0, 0, 0, 0.25, 0, 0, 0.5, 0, 0.25, 0, 0]
input_data_parallel_2 = [0.706181, 0.604714, 0.528406, 0.557752, 0.315469]

# No need to preprocess input data for lists, as they are not Pandas DataFrames

# Load the trained models
model_generic = tf.keras.models.load_model("model_generic.h5")
model_parallel_1 = tf.keras.models.load_model("model_parallel_1.h5")
model_parallel_2 = tf.keras.models.load_model("model_parallel_2.h5")
model_combined = tf.keras.models.load_model("model_combined.h5")

# Convert lists to numpy arrays for compatibility with TensorFlow
X_input_generic = tf.constant([input_data_generic], dtype=tf.float32)
X_input_parallel_1 = tf.constant([input_data_parallel_1], dtype=tf.float32)
X_input_parallel_2 = tf.constant([input_data_parallel_2], dtype=tf.float32)

# Make predictions using individual models
predictions_generic = model_generic.predict(X_input_generic)
predictions_parallel_1 = model_parallel_1.predict(X_input_parallel_1)
predictions_parallel_2 = model_parallel_2.predict(X_input_parallel_2)

# Combine the outputs of parallel AI 1 and AI 2 for the third AI
combined_input = tf.keras.layers.Concatenate()([predictions_parallel_1, predictions_parallel_2])

# Make predictions using the combined model
predictions_combined = model_combined.predict(combined_input)

your_original_labels_for_generic = ['Engineering']
your_original_labels_for_parallel_1 = ['Computer Science and Engineering (including specializations) / Information Technology', 'Electronics and Communication Engineering', 'Mechanical Engineering']
your_original_labels_for_parallel_2 = [ 'Computer Science and Engineering (including specializations) / Information Technology', 'Electronics and Communication Engineering', 'Mechanical Engineering', 'Civil Engineering']
your_original_labels_for_combined = [ 'Computer Science and Engineering (including specializations) / Information Technology', 'Electronics and Communication Engineering', 'Mechanical Engineering', 'Civil Engineering']

# Create and fit LabelEncoders for each model's predictions
label_decoder_generic = LabelEncoder()
label_decoder_generic.fit(your_original_labels_for_generic)  # Replace with your actual labels
label_decoder_parallel_1 = LabelEncoder()
label_decoder_parallel_1.fit(your_original_labels_for_parallel_1)  # Replace with your actual labels
label_decoder_parallel_2 = LabelEncoder()
label_decoder_parallel_2.fit(your_original_labels_for_parallel_2)  # Replace with your actual labels
label_decoder_combined = LabelEncoder()
label_decoder_combined.fit(your_original_labels_for_combined)  # Replace with your actual labels

# Inverse transform the predictions to get the original labels
predicted_labels_generic = label_decoder_generic.inverse_transform(tf.argmax(predictions_generic, axis=1).numpy())
predicted_labels_parallel_1 = label_decoder_parallel_1.inverse_transform(tf.argmax(predictions_parallel_1, axis=1).numpy())
predicted_labels_parallel_2 = label_decoder_parallel_2.inverse_transform(tf.argmax(predictions_parallel_2, axis=1).numpy())
predicted_labels_combined = label_decoder_combined.inverse_transform(tf.argmax(predictions_combined, axis=1).numpy())

# Print or use the predicted labels as needed
print("Predicted Labels - Generic AI:", predicted_labels_generic)
print("Predicted Labels - Parallel AI 1:", predicted_labels_parallel_1)
print("Predicted Labels - Parallel AI 2:", predicted_labels_parallel_2)
print("Predicted Labels - Combined AI:", predicted_labels_combined)

1/1 [==============================] - 0s 92ms/step
Predicted Labels - Generic AI: ['Engineering']
Predicted Labels - Parallel AI 1: ['Electronics and Communication Engineering']
Predicted Labels - Parallel AI 2: ['Computer Science and Engineering (including specializations) / Information Technology']
Predicted Labels - Combined AI: ['Electronics and Communication Engineering']


In [20]:
combined_output

<tf.Tensor: shape=(55, 7), dtype=float32, numpy=
array([[9.08665895e-01, 7.85072744e-02, 1.28267314e-02, 3.23375426e-02,
        9.62293088e-01, 2.99334736e-03, 2.37603881e-03],
       [4.40584630e-01, 5.12869239e-01, 4.65461239e-02, 1.29986137e-01,
        8.17202806e-01, 4.22340818e-02, 1.05769988e-02],
       [6.99215412e-01, 2.72508442e-01, 2.82760914e-02, 8.99858549e-02,
        8.94770086e-01, 1.20007256e-02, 3.24325031e-03],
       [7.77400851e-01, 2.21831277e-01, 7.67842401e-04, 1.06214974e-02,
        9.77967381e-01, 9.87688918e-03, 1.53420400e-03],
       [6.99751794e-01, 2.97398239e-01, 2.85005104e-03, 1.30574867e-01,
        8.53596628e-01, 2.61842133e-03, 1.32102007e-02],
       [9.44211662e-01, 4.08244245e-02, 1.49638383e-02, 3.01475823e-02,
        8.62394989e-01, 1.03872173e-01, 3.58517421e-03],
       [8.79166126e-01, 1.13772824e-01, 7.06104143e-03, 1.30582988e-01,
        8.35585654e-01, 1.79549474e-02, 1.58765037e-02],
       [2.19643191e-02, 5.69939911e-01, 4.080957

In [36]:
import pandas as pd
import numpy as np

# Generate random data for 10 entries with values between 0 and 1
data = np.random.rand(10, 8)

# Create a DataFrame with the generated data
df = pd.DataFrame(data, columns=[0, 1, 2, 3, 4, 5, 6, 7])

# Calculate the maximum value in columns 0, 1, 2, and assign it to column 0
df[3] = df[[0, 1, 2, 3]].max(axis=1)

# Calculate the maximum value in columns 3, 4, 5, and assign it to column 3
df[7] = df[[4, 5, 6, 7]].max(axis=1)

# Save the DataFrame to a CSV file
df.to_csv("generated_data1.csv", index=False)